### SXDG Case

In [72]:
import qiskit

from qiskit_aer import AerSimulator

from rivet_transpiler import get_sinusoids
from rivet_transpiler import get_litmus_circuit

from qiskit_ibm_runtime.fake_provider import fake_backend

from qiskit_ibm_runtime.fake_provider import FakeLimaV2
from qiskit_ibm_runtime.fake_provider import FakeGuadalupeV2
from qiskit_ibm_runtime.fake_provider import FakeMontrealV2

In [2]:
cd ..

/mnt/c/Users/mohor/Jupyter/haiqu


In [3]:
from tests.integration import get_circuits_to_compare
from tests.integration import run_circuits_to_compare
from tests.integration import plot_results_to_compare
from tests.integration import check_delta

In [4]:
# Parameters

OPTIMIZATION_LEVEL = 3

SHOTS_COUNT = 1000
SEED_TRANSPILER = 1234

LAYOUT_METHOD = None
ROUTING_METHOD = None

In [5]:
# def test_transpilation_types_litmus(litmus_circuit, backend):

#     QUBITS_COUNT = litmus_circuit.num_qubits

#     qubits = list(range(QUBITS_COUNT))

#     # Initial State Part

#     initial_state_circuit = qiskit.QuantumCircuit(QUBITS_COUNT)

#     initial_state_circuit.x(0)
#     initial_state_circuit.barrier()

#     # Measurement Part

#     measurement_circuit = qiskit.QuantumCircuit(QUBITS_COUNT, QUBITS_COUNT)
#     measurement_circuit.measure(qubits, qubits)

#     # Circuit Parts

#     CIRCUIT_PARTS = [
#         initial_state_circuit,
#         litmus_circuit,
#         litmus_circuit,
#         measurement_circuit
#     ]

#     # Compare Transpilation Types

#     circuits_to_compare = get_circuits_to_compare(
#         circuit_parts=CIRCUIT_PARTS,
#         backend=backend,
#         layout_method=LAYOUT_METHOD,
#         routing_method=ROUTING_METHOD,
#         seed_transpiler=SEED_TRANSPILER,
#         optimization_level=OPTIMIZATION_LEVEL
#     )

#     results = run_circuits_to_compare(
#         circuits_to_compare=circuits_to_compare,
#         backend=backend,
#         shots_count=SHOTS_COUNT
#     )

#     plot_results_to_compare(results)

#     assert check_delta(results, 0.2), ("Delta Check shows that "
#                                        "counts between transpilation types "
#                                        "differ by more then 20%")

In [71]:
QUBITS_COUNT = 5

litmus_circuit = get_litmus_circuit(QUBITS_COUNT, "Litmus")

litmus_circuit.draw()

┌──────────────┐                    ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(Litmus_0) ├──■─────────────────┤ X ├───░────
            ├──────────────┤┌─┴─┐               └─┬─┘   ░    
Litmus_0_1: ┤ Rz(Litmus_1) ├┤ X ├──■──────────────┼─────░────
            ├──────────────┤└───┘┌─┴─┐            │     ░    
Litmus_0_2: ┤ Rz(Litmus_2) ├─────┤ X ├──■─────────┼─────░────
            ├──────────────┤     └───┘┌─┴─┐       │     ░    
Litmus_0_3: ┤ Rz(Litmus_3) ├──────────┤ X ├──■────┼─────░────
            ├──────────────┤          └───┘┌─┴─┐  │     ░    
Litmus_0_4: ┤ Rz(Litmus_4) ├───────────────┤ X ├──■─────░────
            └──────────────┘               └───┘        ░

In [86]:
fake_backend = FakeMontrealV2()
# fake_backend = FakeGuadalupeV2()
# fake_backend = FakeLimaV2()


backend = AerSimulator.from_backend(fake_backend)

backend

AerSimulator('aer_simulator_from(fake_montreal)'
             noise_model=<NoiseModel on ['reset', 'x', 'cx', 'sx', 'id', 'measure']>)

In [87]:
backend.options.noise_model = None

In [88]:
# test_transpilation_types_litmus(
#     litmus_circuit,
#     backend)

### Disassemble

In [89]:
QUBITS_COUNT = litmus_circuit.num_qubits

qubits = list(range(QUBITS_COUNT))

# Initial State Part

initial_state_circuit = qiskit.QuantumCircuit(QUBITS_COUNT)

initial_state_circuit.x(0)
initial_state_circuit.barrier()

# Measurement Part

measurement_circuit = qiskit.QuantumCircuit(QUBITS_COUNT, QUBITS_COUNT)
measurement_circuit.measure(qubits, qubits)

# Circuit Parts

CIRCUIT_PARTS = [
    initial_state_circuit,
    litmus_circuit,
    litmus_circuit,
    measurement_circuit
]

# Compare Transpilation Types

circuits_to_compare = get_circuits_to_compare(
    circuit_parts=CIRCUIT_PARTS,
    backend=backend,
    layout_method=LAYOUT_METHOD,
    routing_method=ROUTING_METHOD,
    seed_transpiler=SEED_TRANSPILER,
    optimization_level=OPTIMIZATION_LEVEL
)

# results = run_circuits_to_compare(
#     circuits_to_compare=circuits_to_compare,
#     backend=backend,
#     shots_count=SHOTS_COUNT
# )

In [90]:
circuits_to_compare

{'Combined': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7ff9b467ba30>,
 'Blocks': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7ff9b4666110>,
 'Chain': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7ff9bc4ae710>,
 'Right': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7ff9b60d5de0>,
 'Left': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7ff9bc172500>}

In [91]:
circuit = circuits_to_compare['Left']

circuit.draw(fold=-1, idle_wires=False)

┌───┐                                                                                                                                                ░          ┌─┐   
  ancilla_4 -> 4 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░──────────┤M├───
                                                                                                                                                                                                                                                                                                                                                                                             ┌───┐     ┌─┴─┐└─┬─┘┌─┴─┐                                                                                                                                           ░          └╥┘   
  ancilla_7 -> 7 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■──┤ X ├──■──┤ X ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╫────
                                                                                                                                                                                                                                                                                                                                                                              ┌───┐     ┌─┴─┐└─┬─┘┌─┴─┐└───┘     └───┘                                                                                                                                                       ║    
ancilla_10 -> 10 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■──┤ X ├──■──┤ X ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╫────
                                                                                                                                       ┌───┐                                         ░                                                                                                                            ┌───┐                                        ┌───┐     ┌─┴─┐└─┬─┘┌─┴─┐└───┘     └───┘                                                                                                                                                                      ║    
ancilla_12 -> 12 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■──────────────────────────────────────░─────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────┤ X ├──■────────────────────────■─────────■──┤ X ├──■──┤ X ├──■──┤ X ├───────────────────────────────────────────────────────────────────────────────

In [85]:
# Assign Parameters

for index, parameter in enumerate(circuit.parameters):

    circuit.assign_parameters({parameter: index}, inplace=True)

# Backend

if backend is None:
    run_backend = AerSimulator()
    run_circuit = transpile(circuit, run_backend)
else:
    run_backend = backend
    run_circuit = circuit

# Run

job = run_backend.run(run_circuit, shots=SHOTS_COUNT)

counts = job.result().get_counts()

counts

{'01010': 1000}